# Random Forests

## Import Statements

In [58]:
import pandas as pd
import numpy as np

#import matplotlib as plt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from rdkit import Chem

# FEATURES
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem # For Morgan Fingerprint (Circular Fingerprints)
from rdkit.Chem import MACCSkeys # For MACCS keys

# SCALING DATA
from sklearn.preprocessing import scale

# For splitting data into training and test sets.
from sklearn.model_selection import train_test_split

# For processing how well our methods have classified our data
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score,roc_auc_score

# Principal Component Analysis (PCA)
from sklearn.decomposition import PCA

# Logistic Regression ML Model
from sklearn.linear_model import LogisticRegression

#
from sklearn.ensemble import RandomForestClassifier

# Cross Validation
from sklearn.model_selection import cross_val_score

#CV
from sklearn.model_selection import cross_validate

## User Defined Helper Functions

In [73]:
####################################################################
###
###    User Defined Helper Functions
###
####################################################################

# FUNCTION LOAD_DATA
# Load the data from the csv file in the [data] folder. We separate 
# the molecule data from the "explosive" labes and return the labels
# an a molecules from SMILES representation for each molecule in the 
# dataset
def load_data(data_file):
    mol_df = pd.read_csv(data_file)

    smiles = np.array(mol_df['smiles'])
    labels = np.array(mol_df['labels']) 
    
    mols = np.array([Chem.MolFromSmiles(smile) for smile in smiles])
    
    return mols, labels

# FUNCTION GEN_FPRINTS
# Generate molecule fingerprints for each molecule in an input list
def gen_fprints(mols):
    f_prints = np.array([Chem.RDKFingerprint(mol) for mol in mols])  
    return f_prints

# FUNCTION gen_MACCS 
# Generate the MACCS keys for each molecule in an input list of molecules
def gen_MACCS(mols):
    MACCS_keys = np.array([MACCSkeys.GenMACCSKeys(mol) for mol in mols])
    return MACCS_keys

# FUNCTION gen_morgan_prints
# Generates Morgan finger prints for each molecule in an input list of molecules
def gen_morgan_prints(mols,radius):
    morgan_prints = np.array([AllChem.GetMorganFingerprintAsBitVect(mol,radius,nBits=1024) for mol in mols])
    return morgan_prints

# FUNCTION CrossValidation
# Runs Cross Validation and outputs the results in a dataframe
def CrossValidation(ML_model,X,Y):
    model_scores = []
    model_scores.append(cross_validate(ML_model, X , Y , cv=5, scoring=('recall','accuracy', 'f1', 'roc_auc','precision'), return_train_score=True))
    models_df = pd.DataFrame(model_scores, columns = ['fit_time','score_time','test_recall','train_recall','test_accuracy','train_accuracy','test_f1','train_f1','test_roc_auc','train_roc_auc','test_precision','train_precision'])
    av_column = models_df.mean(axis = 0)
    return pd.DataFrame(av_column)

# FUNCTION Output
# Prints out the Scores of the model
def Output(pred,label_test):
    # Model Generalizability Analysis
    accuracy = accuracy_score(label_test, pred)
    conf_matrix = confusion_matrix(label_test, pred)
    F1Score = f1_score(label_test,pred)
    roc_auc = roc_auc_score(label_test,pred)
    recall = recall_score(label_test,pred)

    print('\033[1m' + 'Confusion Matrix' + '\033[0m') # printing in bold
    print(conf_matrix)
    
    print('\033[1m' + '\nRecall' + '\033[0m')
    print(recall)  

    print('\033[1m' + '\nAccuracy' + '\033[0m')
    print(accuracy)  
    
    print('\033[1m' + '\nF1 Score' + '\033[0m')
    print(F1Score)

    print('\033[1m' + '\nROC AUC Score' + '\033[0m')
    print(roc_auc)

    
def getKeyFromBond(bond):
    atom1 = int(bond.GetBeginAtom().GetAtomicNum())
    atom2 = int(bond.GetEndAtom().GetAtomicNum())

    if atom1 > atom2:
        atom1, atom2 = atom2, atom1

    bondType = int(bond.GetBondTypeAsDouble() * 2 - 2)
    key = atom1 | (atom2 << 8) | (bondType << 16)

    return key

def formatMolecule(ID):
    pt = Chem.GetPeriodicTable()
    atom1 = pt.GetElementSymbol(int(ID & 255))
    atom2 = pt.GetElementSymbol(int((ID >> 8) & 255))
    bondType = ['-', ':', '=', 'err', '#'][(ID >> 16) & 255]
    return '%s%s%s' % (atom1,bondType,atom2)

def gen_nathan_prints(mols):
    # this dictionary is responsible for assigning a unique ID (index) to
    # every unique bond.  The bonds are assigned incrementing IDs as discovered
    bondIDs = dict()
    numUniqueBonds = 0
    
    for molecule in mols:
        for bond in molecule.GetBonds():
            key = getKeyFromBond(bond)
            if key not in bondIDs:
                bondIDs[key] = numUniqueBonds
                numUniqueBonds += 1
    
    # list of numpy byte arrays representing the feature vector of each molecule
    fingerprints = []

    for molecule in mols:
        # One molecule has 105 carbon-carbon single bonds, so the fingerprint
        # format is set to preserve up to that many occurances of any unique bond
            
        # each fingerprint requires 1 byte to store the count of each unique bond type
        # plus the extra 2 bytes store the molecule's molar mass
        fingerprint = np.zeros(numUniqueBonds + 2, np.uint8)
        
        for bond in molecule.GetBonds():
            key = getKeyFromBond(bond)
            index = bondIDs[key]
            fingerprint[index] += 1
        
        # the heaviest molecule in our dataset weights 3431.9089999999887 g/mol
        # encode the weight with 1/5 increments of fractional value
        # in the last 2 bytes of the finger print / feature vector
        weight = Descriptors.MolWt(molecule)
        intWeight = round(weight * 5)
        fingerprint[numUniqueBonds] = intWeight & 255
        fingerprint[numUniqueBonds + 1] = (intWeight >> 8) & 255
        
        fingerprints.append(fingerprint)
    
    # create a mapping from bond ID to bond key to aid feature importance
    sourceMap = np.zeros(numUniqueBonds, np.uint32)
    for k, v in bondIDs.items():
        sourceMap[v] = k
    
    return fingerprints, sourceMap

## Loading Data 

In [50]:
data_file = 'molecule_data.csv'
mols, labels = load_data(data_file)

## Using Morgan Features

In [51]:
# load different molecular features separately
##############################################
#rdk_features    = gen_fprints(mols)
#maccs_features  = gen_MACCS(mols)
morgan_features = gen_morgan_prints(mols,radius=16)

In [52]:
# Split data for [training] and [testing]
morgan_train, morgan_test, label_train, label_test = train_test_split(morgan_features, labels, \
                                                                test_size=0.3, shuffle=True)

In [53]:
######################
# Principal Components
######################

# Determine principal components using [training data] and apply the transformation to the [test data]
PC_morgan_proj = PCA(n_components=961)
morgan_proj_train = PC_morgan_proj.fit_transform(morgan_train)
morgan_proj_test = PC_morgan_proj.transform(morgan_test)
morgan_proj_features = PC_morgan_proj.fit_transform(morgan_features)

### Random Forests with PCA

In [55]:
#########################################################################
#
# Random Forests with PCA
#
#########################################################################
# Train Random Forests Classifier using PC transformed Morgan features
RF_PCA = RandomForestClassifier(n_estimators = 1000,criterion='gini',bootstrap = True, warm_start = False)
RF = RF_PCA.fit(morgan_train,label_train)
pred1 = RF.predict(morgan_test)

#### Scores

In [63]:
Scores1 = Output(pred1,label_test)
Scores1

Confusion Matrix
[[1179    1]
 [   8   69]]

Recall
0.8961038961038961

Accuracy
0.9928400954653938

F1 Score
0.9387755102040817

ROC AUC Score
0.9476282192383887


#### Cross Validation

In [64]:
M_Model1_df = CrossValidation(RF_PCA,morgan_proj_features,labels)
M_Model1_df

,0,ID
fit_time,120.379936,1
score_time,1.185863,2
test_recall,0.358491,3
train_recall,1.000000,4
test_accuracy,0.959427,5
train_accuracy,1.000000,6
test_f1,0.527778,7
train_f1,1.000000,8
test_roc_auc,0.975075,9
train_roc_auc,1.000000,10


### Random Forests [WITHOUT] PCA

In [67]:
#########################################################################
#
# Random Forests [WITHOUT] PCA
#
#########################################################################
# Train Random Forests model using PC transformed Morgan features

RF_ = RandomForestClassifier(n_estimators = 1000,bootstrap = True, warm_start = False)
rf = RF_.fit(morgan_train,label_train)
pred2 = rf.predict(morgan_test)

#### Scores

In [25]:
Scores2 = Output(pred2,label_test)
Scores2

Confusion Matrix
[[1180    3]
 [  10   64]]

Recall
0.8648648648648649

Accuracy
0.9896579156722355

F1 Score
0.9078014184397162

ROC AUC Score
0.9311644696260081


#### Cross Validation

In [68]:
M_Model2_df = CrossValidation(RF_,morgan_features,labels)
M_Model2_df

,0,ID
fit_time,18.157396,1
score_time,0.934957,2
test_recall,0.830189,3
train_recall,1.000000,4
test_accuracy,0.986874,5
train_accuracy,1.000000,6
test_f1,0.888889,7
train_f1,1.000000,8
test_roc_auc,0.980675,9
train_roc_auc,1.000000,10


## Using MACCS Features

In [31]:
# load different molecular features separately
##############################################
#rdk_features    = gen_fprints(mols)
maccs_features  = gen_MACCS(mols)
#morgan_features = gen_morgan_prints(mols,radius=18)

In [32]:
# Split data for [training] and [testing]
maccs_train, maccs_test, label_train, label_test = train_test_split(maccs_features, labels, \
                                                                test_size=0.3, shuffle=True)

In [33]:
######################
# Principal Components
######################

# Determine principal components using [training data] and apply the transformation to the [test data]
PC_maccs_proj = PCA(n_components=144)
maccs_proj_train = PC_maccs_proj.fit_transform(maccs_train)
maccs_proj_test = PC_maccs_proj.transform(maccs_test)
maccs_proj_features = PC_maccs_proj.fit_transform(maccs_features)

### Random Forests with PCA

In [34]:
#########################################################################
#
# Random Forests with PCA
#
#########################################################################
# Train Random Forests Classifier using PC transformed Morgan features

RF_PCA = RandomForestClassifier(n_estimators = 100,criterion='gini',bootstrap = True, warm_start = False)
rf = RF_PCA.fit(maccs_train,label_train)
pred1 = rf.predict(maccs_test)

#### Scores

In [35]:
Scores1 = Output(pred1,label_test)
Scores1

Confusion Matrix
[[1185    1]
 [   8   63]]

Recall
0.8873239436619719

Accuracy
0.9928400954653938

F1 Score
0.9333333333333333

ROC AUC Score
0.9432403866707837


#### Cross Validation

In [ ]:
MAC_Model1_df = CrossValidation(RF_PCA,maccs_proj_features,labels)
MAC_Model1_df

### Random Forests [WITHOUT] PCA

In [36]:
#########################################################################
#
# Random Forests [WITHOUT] PCA
#
#########################################################################
# Train Random Forests model using PC transformed Morgan features

RF_ = RandomForestClassifier(n_estimators = 100,criterion='gini',bootstrap = True, warm_start = False)
rf = RF_.fit(maccs_train,label_train)
pred2 = rf.predict(maccs_test)

#### Scores

In [37]:
Scores2 = Output(pred2,label_test)
Scores2

Confusion Matrix
[[1185    1]
 [   8   63]]

Recall
0.8873239436619719

Accuracy
0.9928400954653938

F1 Score
0.9333333333333333

ROC AUC Score
0.9432403866707837


#### Cross Validation

In [ ]:
MAC_Model2_df = CrossValidation(RF_,maccs_features,labels)
MAC_Model2_df

## Using RDKit Features

In [39]:
# load different molecular features separately
##############################################
rdk_features    = gen_fprints(mols)
#maccs_features  = gen_MACCS(mols)
#morgan_features = gen_morgan_prints(mols,radius=18)

In [40]:
# Split data for [training] and [testing]
rdk_train, rdk_test, label_train, label_test = train_test_split(rdk_features, labels, \
                                                                test_size=0.3, shuffle=True)

In [41]:
######################
# Principal Components
######################

# Determine principal components using [training data] and apply the transformation to the [test data]

PC_rdk_proj = PCA(n_components=1570)
rdk_proj_train = PC_rdk_proj.fit_transform(rdk_train)
rdk_proj_test = PC_rdk_proj.transform(rdk_test)
rdk_proj_features = PC_rdk_proj.fit_transform(rdk_features)

### Random Forests with PCA

In [42]:
#########################################################################
#
# Random Forests with PCA
#
#########################################################################
# Train Random Forests Classifier using PC transformed Morgan features

RF_PCA = RandomForestClassifier(n_estimators = 100,criterion='gini',bootstrap = True, warm_start = False)
rf = RF_PCA.fit(rdk_train,label_train)
pred1 = rf.predict(rdk_test)

#### Scores

In [43]:
Scores1 = Output(pred1,label_test)
Scores1

Confusion Matrix
[[1177    4]
 [  15   61]]

Recall
0.8026315789473685

Accuracy
0.984884645982498

F1 Score
0.8652482269503547

ROC AUC Score
0.8996223093720754


#### Cross Validation

In [ ]:
RDK_Model1_df = CrossValidation(RF_PCA,rdk_proj_features,labels)
RDK_Model1_df

### Random Forests [WITHOUT] PCA

In [44]:
#########################################################################
#
# Random Forests [WITHOUT] PCA
#
#########################################################################
# Train Random Forests model using PC transformed Morgan features

RF_ = RandomForestClassifier(n_estimators = 100,criterion='gini',bootstrap = True, warm_start = False)
rf = RF_.fit(rdk_train,label_train)
pred2 = rf.predict(rdk_test)

#### Scores

In [45]:
Scores2 = Output(pred2,label_test)
Scores2

Confusion Matrix
[[1179    2]
 [  16   60]]

Recall
0.7894736842105263

Accuracy
0.9856801909307876

F1 Score
0.8695652173913043

ROC AUC Score
0.8938901020544587


#### Cross Validation

In [ ]:
RDK_Model2_df = CrossValidation(RF_,rdk_features,labels)
RDK_Model2_df

## Using Nathan Features

In [ ]:
# load different molecular features separately
##############################################
#rdk_features    = gen_fprints(mols)
#maccs_features  = gen_MACCS(mols)
#morgan_features = gen_morgan_prints(mols,radius=18)
nathan_features, bondIDSourceMap = gen_nathan_prints(mols)

In [ ]:
# Split data for [training] and [testing]
nathan_train, nathan_test, label_train, label_test = train_test_split(nathan_features, labels, \
                                                                test_size=0.3, shuffle=True)

In [ ]:
######################
# Principal Components
######################

# Determine principal components using [training data] and apply the transformation to the [test data]
PC_nathan_proj = PCA(n_components = 3)
nathan_proj_train = PC_nathan_proj.fit_transform(nathan_train)
nathan_proj_test = PC_nathan_proj.transform(nathan_test)
nathan_proj_features = PC_nathan_proj.fit_transform(nathan_features)

### Random Forests with PCA

In [ ]:
#########################################################################
#
# Random Forests with PCA
#
#########################################################################
# Train Random Forests Classifier using PC transformed Morgan features

RF_PCA = RandomForestClassifier(n_estimators = 100,criterion='gini',bootstrap = True, warm_start = False)
rf = RF_PCA.fit(nathan_train,label_train)
pred1 = rf.predict(nathan_test)

#### Scores

In [ ]:
Scores1 = Output(pred1,label_test)
Scores1

#### Cross Validation

In [47]:
N_Model1_df = CrossValidation(RF_PCA,nathan_proj_features,labels)
N_Model1_df

NameError: name 'AdaModel_PCA' is not defined

### Random Forests [WITHOUT] PCA

In [ ]:
#########################################################################
#
# Random Forests [WITHOUT] PCA
#
#########################################################################
# Train Random Forests model using PC transformed Morgan features

RF_ = RandomForestClassifier(n_estimators = 100,criterion='gini',bootstrap = True, warm_start = False)
rf = RF_.fit(nathan_train,label_train)
pred2 = rf.predict(nathan_test)

#### Scores

In [ ]:
Scores2 = Output(pred2,label_test)
Scores2

#### Cross Validation

In [ ]:
N_Model2_df = CrossValidation(RF_,nathan_features,labels)
N_Model2_df

## FULL Dataframe

In [ ]:
result = pd.concat([M_Model1_df, M_Model2_df,MAC_Model1_df,MAC_Model2_df,RDK_Model1_df,RDK_Model2_df,N_Model1_df,N_Model2_df], axis=1, join='inner')
result.columns = ['M_Model1_df','M_Model2_df','MAC_Model1_df','MAC_Model2_df','RDK_Model1_df','RDK_Model2_df',"N_Model1_df","N_Model2_df"]
result = result.transpose()
result.to_csv("RF_Result.csv")
result